In [195]:
import sys
import pandas as pd
import numpy as np
import datetime as dt
# 프로젝트 루트 직접 지정
sys.path.insert(0, r"c:\Users\tjrrj\vscode\doritest")

# 코드

In [181]:
"""
구글 드라이브 엑셀 파일 추출 함수

사용법:
    df = extract_excel_from_gdrive(
        url="https://docs.google.com/spreadsheets/d/파일ID/edit...",
        sheet_name="시트이름"  # 선택사항, 없으면 첫 번째 시트
    )

주의사항:
    - 구글 드라이브에 있는 엑셀 파일(.xlsx)을 읽을 때 사용
    - 서비스 계정을 파일 공유에 추가해야 함
    - 순수 구글 스프레드시트는 extract_gsheet() 사용
"""
import re
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import pandas as pd

def extract_excel_from_gdrive(
    url: str = None, 
    file_id: str = None,
    sheet_name: str = None, 
    credentials_path: str = None
):
    """구글 드라이브에 있는 엑셀 파일을 다운로드하여 읽기
    
    Args:
        url: 구글 드라이브 파일 URL
        file_id: 파일 ID (url 대신 사용 가능)
        sheet_name: 읽을 워크시트(탭) 이름. 없으면 첫 번째 시트
        credentials_path: 서비스 계정 JSON 파일 경로
    
    Returns:
        DataFrame: 읽어온 데이터
    """
    credentials_path = credentials_path or r'c:\Users\민준\doridang_airflow\config\glowing-palace-465904-h6-7f82df929812.json'
    
    # URL에서 파일 ID 추출
    if url:
        pattern = r"/d/([a-zA-Z0-9_-]+)"
        match = re.search(pattern, url)
        if not match:
            raise ValueError("올바른 구글 드라이브 URL 형식이 아닙니다.")
        file_id = match.group(1)
    
    if not file_id:
        raise ValueError("url 또는 file_id 중 하나는 반드시 필요합니다.")
    
    # 인증
    creds = service_account.Credentials.from_service_account_file(
        credentials_path,
        scopes=['https://www.googleapis.com/auth/drive.readonly']
    )
    
    # Drive API 클라이언트 생성
    service = build('drive', 'v3', credentials=creds)
    
    # 파일 다운로드
    request = service.files().get_media(fileId=file_id)
    file_buffer = io.BytesIO()
    downloader = MediaIoBaseDownload(file_buffer, request)
    
    done = False
    while not done:
        status, done = downloader.next_chunk()
    
    file_buffer.seek(0)
    
    # 엑셀 파일 읽기
    if sheet_name:
        return pd.read_excel(file_buffer, sheet_name=sheet_name)
    else:
        return pd.read_excel(file_buffer)

# 사용 예시
df = extract_excel_from_gdrive(
    url="https://docs.google.com/spreadsheets/d/1a6-20U1-FYCQEfbOOVSDG3M0q6G2me5f/edit?usp=sharing&ouid=116296472165065607173&rtpof=true&sd=true",
    sheet_name="가맹점리스트"
)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,심성준 이사,Unnamed: 6,Unnamed: 7,3,황대성 대리,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,NaN,NaN,NaN,NaN,NaN,김덕기 과장,NaN,NaN,22,이병두 과장,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,오픈\n순서,호점,매장명,사업자 번호,점주명,담당 S.V,담당SV,변경 \n담당 SV,계약체결일,...,요기요ID,요기요PW,쿠팡이츠ID,쿠팡이츠PW,땡겨요ID,땡겨요PW,토더 ID,토더PW,네이버 ID,네이버 pw
2,NaN,1,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,김덕기 과장,김덕기 과장,NaN,...,mjps9142,mjps$0214,mjps9142,mjps$0214,mjps9142,mjps%0214,doridang100001,abcd9142,NaN,NaN
3,NaN,2,2,도리당 송파점,332-20-02029,홍창현,심성준 이사,이병두 과장,이병두 과장,NaN,...,songdori,ehfl0301!,justmarket44,ehfl0301!,songdori,ehfl0301!,doridang100002,abcd6048,NaN,NaN
4,NaN,NaN,NaN,도리당 교대점,NaN,정수미,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,NaN,297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,NaN,298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,NaN,299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
df = df.drop(columns=["Unnamed: 0"])

In [183]:
header_row_idx = 1
df.columns = df.iloc[header_row_idx]


In [184]:
df = df.rename(columns=({
    '오픈\n순서' : '오픈순서',
    '변경 \n담당 SV' : '변경담당SV'
}))
df = df[2:]
cond1 = ~df["사업자 번호"].isnull()
df = df.loc[cond1]

In [185]:
df['상세주소'] = (
    df['주소'].astype(str)
    # 개행/HTML 특수문자 정리
    .str.replace('\r', ' ', regex=False)
    .str.replace('\n', ' ', regex=False)
    .str.replace('&gt;', '>', regex=False)
    # (이사후 주소) 뒤 텍스트만 추출
    .str.extract(r'\(이사후 주소\)\s*(.+)', expand=False)
    # 없으면 원본에서 (이사전 주소) 라벨 제거한 값으로 대체
    .fillna(
        df['주소'].astype(str)
        .str.replace(r'\(이사전 주소\)\s*', '', regex=True)
        .str.replace('\r', ' ', regex=False)
        .str.replace('\n', ' ', regex=False)
        .str.replace('&gt;', '>', regex=False)
    )
    .str.strip()
)

In [186]:
df_col = ['오픈순서', '호점', '매장명', '사업자 번호', '점주명', '변경담당SV',
       '실오픈일', '상세주소', '배달의 민족ID', '배달의 민족PW', '요기요ID', '요기요PW', '쿠팡이츠ID', '쿠팡이츠PW',
       '땡겨요ID', '땡겨요PW', '토더 ID', '토더PW', '네이버 ID', '네이버 pw']

gsheet_store_list = df[df_col]


In [187]:
gsheet_store_list["광역"] = gsheet_store_list['상세주소'].str.split(' ').str[0].str[:2]
gsheet_store_list["시군구"] = gsheet_store_list['상세주소'].str.split(' ').str[1]
gsheet_store_list["읍면동"] = gsheet_store_list['상세주소'].str.split(' ').str[2]
col2 = ['오픈순서', '호점', '매장명', '사업자 번호', '점주명', '변경담당SV', '실오픈일', 
       '상세주소', '광역', '시군구', '읍면동',
       '배달의 민족ID', '배달의 민족PW', '요기요ID', '요기요PW', '쿠팡이츠ID', '쿠팡이츠PW', '땡겨요ID',
       '땡겨요PW', '토더 ID', '토더PW', '네이버 ID', '네이버 pw']

C:\Users\민준\AppData\Local\Temp\ipykernel_26772\2988297795.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gsheet_store_list["광역"] = gsheet_store_list['상세주소'].str.split(' ').str[0].str[:2]
C:\Users\민준\AppData\Local\Temp\ipykernel_26772\2988297795.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gsheet_store_list["시군구"] = gsheet_store_list['상세주소'].str.split(' ').str[1]
C:\Users\민준\AppData\Local\Temp\ipykernel_26772\2988297795.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [188]:
gsheet_store_list = gsheet_store_list[col2]
gsheet_store_list.head()

1,오픈순서,호점,매장명,사업자 번호,점주명,변경담당SV,실오픈일,상세주소,광역,시군구,...,요기요ID,요기요PW,쿠팡이츠ID,쿠팡이츠PW,땡겨요ID,땡겨요PW,토더 ID,토더PW,네이버 ID,네이버 pw
2,1,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18 00:00:00,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,...,mjps9142,mjps$0214,mjps9142,mjps$0214,mjps9142,mjps%0214,doridang100001,abcd9142,NaN,NaN
3,2,2,도리당 송파점,332-20-02029,홍창현,이병두 과장,2023-09-08 00:00:00,서울시 송파구 송파대로 37길 70 (석촌동 254-5) 1층 11호,서울,송파구,...,songdori,ehfl0301!,justmarket44,ehfl0301!,songdori,ehfl0301!,doridang100002,abcd6048,NaN,NaN
5,3,2025-03-01 00:00:00,도리당 교대점,230-10-02699,박미미,심성준 이사,2024-06-07 00:00:00,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,...,jkja905,rudwk0905!,jkja905,rudwk0905!,NaN,NaN,doridang100003,abcd9681,NaN,NaN
6,4,4,도리당 강동점,378-28-01624,박진후,심성준 이사,2023-11-06 00:00:00,"서울특별시 강동구 상암로 126, 1층 104호",서울,강동구,...,24108814,kolakola1!,pak0145,kolakola1!,pak0145,kola01624!,doridang100004,abcd8814,NaN,NaN
7,5,5,도리당 법흥리점,141-05-55913,손소연,김덕기 과장,2023-11-21 00:00:00,"경기도 파주시 탄현면 약산로 74, 1층",경기,파주시,...,hjh7312,7312hjhhjh#,NaN,NaN,NaN,NaN,doridang100005,abcd4696,NaN,NaN


In [189]:
# 플랫폼별 컬럼 매핑
platform_cols = {
    '배달의 민족': ('배달의 민족ID', '배달의 민족PW'),
    '요기요': ('요기요ID', '요기요PW'),
    '쿠팡이츠': ('쿠팡이츠ID', '쿠팡이츠PW'),
    '땡겨요': ('땡겨요ID', '땡겨요PW'),
    '토더': ('토더 ID', '토더PW'),
    '네이버': ('네이버 ID', '네이버 pw')
}

# 매장 기본 정보
keep_cols = ['오픈순서', '호점', '매장명', '사업자 번호', '점주명', "변경담당SV", '실오픈일', '상세주소', '광역', '시군구', '읍면동']

rows = []
for _, r in gsheet_store_list.iterrows():
    base = {c: r[c] for c in keep_cols}
    for platform, (id_col, pw_col) in platform_cols.items():
        uid = r.get(id_col)
        pwd = r.get(pw_col)
        if pd.notna(uid) or pd.notna(pwd):  # 둘 중 하나라도 값이 있으면 추가
            rows.append({**base, '플랫폼': platform, '계정ID': uid, '계정PW': pwd})

# 새 데이터프레임 생성
accounts_long = pd.DataFrame(rows)

# 결과 확인
gsheet_store_list = accounts_long
gsheet_store_list


,오픈순서,호점,매장명,사업자 번호,점주명,변경담당SV,실오픈일,상세주소,광역,시군구,읍면동,플랫폼,계정ID,계정PW
0,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,배달의 민족,mjps9142,mjps%0214
1,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,요기요,mjps9142,mjps$0214
2,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,쿠팡이츠,mjps9142,mjps$0214
3,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,땡겨요,mjps9142,mjps%0214
4,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,토더,doridang100001,abcd9142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,75.0,65,도리당 김해인제대점,229-229-01649,이은아,이병두 과장,2025-12-18,경남 김해시 활천로 267번길 6-30,경남,김해시,활천로,쿠팡이츠,ss813847,7642shin@@
344,75.0,65,도리당 김해인제대점,229-229-01649,이은아,이병두 과장,2025-12-18,경남 김해시 활천로 267번길 6-30,경남,김해시,활천로,땡겨요,ss813847,7642shin@@
345,76.0,67,도리당 오산시청점,414-01-72506,김수연,황대성 대리,2026-01-08,"경기도 오산시 운천로 80-17, 1층 101호(오산동 , 명진빌딩)",경기,오산시,운천로,배달의 민족,a4140172506,b4140172506
346,76.0,67,도리당 오산시청점,414-01-72506,김수연,황대성 대리,2026-01-08,"경기도 오산시 운천로 80-17, 1층 101호(오산동 , 명진빌딩)",경기,오산시,운천로,요기요,a4140172506,b$!40172506


In [190]:
gsheet_store_list["collected_at"] = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
gsheet_store_list


,오픈순서,호점,매장명,사업자 번호,점주명,변경담당SV,실오픈일,상세주소,광역,시군구,읍면동,플랫폼,계정ID,계정PW,collected_at
0,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,배달의 민족,mjps9142,mjps%0214,2026-01-02 10:26:36
1,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,요기요,mjps9142,mjps$0214,2026-01-02 10:26:36
2,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,쿠팡이츠,mjps9142,mjps$0214,2026-01-02 10:26:36
3,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,땡겨요,mjps9142,mjps%0214,2026-01-02 10:26:36
4,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,토더,doridang100001,abcd9142,2026-01-02 10:26:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,75.0,65,도리당 김해인제대점,229-229-01649,이은아,이병두 과장,2025-12-18,경남 김해시 활천로 267번길 6-30,경남,김해시,활천로,쿠팡이츠,ss813847,7642shin@@,2026-01-02 10:26:36
344,75.0,65,도리당 김해인제대점,229-229-01649,이은아,이병두 과장,2025-12-18,경남 김해시 활천로 267번길 6-30,경남,김해시,활천로,땡겨요,ss813847,7642shin@@,2026-01-02 10:26:36
345,76.0,67,도리당 오산시청점,414-01-72506,김수연,황대성 대리,2026-01-08,"경기도 오산시 운천로 80-17, 1층 101호(오산동 , 명진빌딩)",경기,오산시,운천로,배달의 민족,a4140172506,b4140172506,2026-01-02 10:26:36
346,76.0,67,도리당 오산시청점,414-01-72506,김수연,황대성 대리,2026-01-08,"경기도 오산시 운천로 80-17, 1층 101호(오산동 , 명진빌딩)",경기,오산시,운천로,요기요,a4140172506,b$!40172506,2026-01-02 10:26:36


In [194]:

# 조건에 맞는 행에만 email 값 추가
gsheet_store_list.loc[gsheet_store_list["매장명"] == "도리당 가락점", "email"] = "a17019@kakao.com"
gsheet_store_list.loc[gsheet_store_list["매장명"] == "도리당 가락점", "쿠팡_매장명"] = "닭도리탕 전문 도리당 가락점"
gsheet_store_list.loc[gsheet_store_list["매장명"] == "도리당 가락점", "배민_매장명"] = "[음식배달] 닭도리탕 전문 도리당 가락점"
gsheet_store_list.rename(columns={"변경담당SV": "담당자"}, inplace=True)
gsheet_store_list

,오픈순서,호점,매장명,사업자 번호,점주명,담당자,실오픈일,상세주소,광역,시군구,읍면동,플랫폼,계정ID,계정PW,collected_at,email,쿠팡_매장명,배민_매장명
0,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,배달의 민족,mjps9142,mjps%0214,2026-01-02 10:26:36,NaN,NaN,NaN
1,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,요기요,mjps9142,mjps$0214,2026-01-02 10:26:36,NaN,NaN,NaN
2,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,쿠팡이츠,mjps9142,mjps$0214,2026-01-02 10:26:36,NaN,NaN,NaN
3,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,땡겨요,mjps9142,mjps%0214,2026-01-02 10:26:36,NaN,NaN,NaN
4,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,토더,doridang100001,abcd9142,2026-01-02 10:26:36,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,75.0,65,도리당 김해인제대점,229-229-01649,이은아,이병두 과장,2025-12-18,경남 김해시 활천로 267번길 6-30,경남,김해시,활천로,쿠팡이츠,ss813847,7642shin@@,2026-01-02 10:26:36,NaN,NaN,NaN
344,75.0,65,도리당 김해인제대점,229-229-01649,이은아,이병두 과장,2025-12-18,경남 김해시 활천로 267번길 6-30,경남,김해시,활천로,땡겨요,ss813847,7642shin@@,2026-01-02 10:26:36,NaN,NaN,NaN
345,76.0,67,도리당 오산시청점,414-01-72506,김수연,황대성 대리,2026-01-08,"경기도 오산시 운천로 80-17, 1층 101호(오산동 , 명진빌딩)",경기,오산시,운천로,배달의 민족,a4140172506,b4140172506,2026-01-02 10:26:36,NaN,NaN,NaN
346,76.0,67,도리당 오산시청점,414-01-72506,김수연,황대성 대리,2026-01-08,"경기도 오산시 운천로 80-17, 1층 101호(오산동 , 명진빌딩)",경기,오산시,운천로,요기요,a4140172506,b$!40172506,2026-01-02 10:26:36,NaN,NaN,NaN


In [198]:
gsheet_store_list.to_csv(r"E:\d_down\del.csv", index=False, encoding='cp949')

In [3]:
import pandas as pd
df = pd.read_parquet(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\temp\orders_with_stores_20260105.parquet")

In [ ]:
import pandas as pd
df = pd.read_parquet(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\temp\baemin_parquet_path_20260106.parquet")
df